# Root level

In [1]:
%%writefile package.json
{
  "name": "nfl-kicker-assessment",
  "private": true,
  "scripts": {
    "start": "npm start --prefix src/backend",
    "dev": "concurrently \"npm start --prefix src/backend\" \"npm run dev --prefix src/frontend\"",
    "dev:api": "cross-env PYTHONPATH=src uvicorn backend.ML.model_api.main:app --host 0.0.0.0 --port 8000 --reload",
    "dev:all": "concurrently \"cross-env MODEL_API_URL=http://127.0.0.1:8000 npm start --prefix src/backend\" \"npm run dev:api\" \"npm run dev --prefix src/frontend\"",
    "build": "npm run build --prefix src/frontend",
    "preview": "npm run build && npx serve -s src/frontend/dist",
    "ci:validate": "node scripts/ci-validate.js",
    "test": "npm test --prefix src/backend && npm test --prefix src/frontend"
  },
  "devDependencies": {
    "concurrently": "^8.2.0",
    "cross-env": "^7.0.3"
  }
}



Overwriting package.json


# Validation

In [2]:
%%writefile render.yaml
services:
  - type: web
    name: nfl-kicker-api
    env: node
    plan: free
    rootDir: src/backend
    buildCommand: "npm ci --omit=dev"
    startCommand: "node server.js"
    healthCheckPath: /api/ping    # Render watches this
    autoDeploy: true
    envVars:
      - key: CORS_ORIGIN
        value: "https://<your-site>.netlify.app"
      - key: NODE_ENV
        value: production


Overwriting render.yaml


In [3]:
%%writefile netlify.toml
[build]
  command = "npm run build --prefix src/frontend"
  publish = "src/frontend/dist"

[[redirects]]
  from = "/*"
  to   = "/index.html"
  status = 200


Overwriting netlify.toml


# github actions


In [4]:
%%writefile .github/workflows/netlify.yml
name: Netlify Deploy
on: { push: { branches: [ main ] } }

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with: { node-version: '18' }

      # Validate environment variables
      - name: Validate environment variables
        run: node scripts/ci-validate.js
        env:
          VITE_API_URL: ${{ secrets.VITE_API_URL }}
          CORS_ORIGIN: ${{ secrets.CORS_ORIGIN }}

      # Build and deploy
      - run: npm ci --prefix src/frontend
      - run: npm run build --prefix src/frontend
      - uses: netlify/actions/cli@master
        env:
          NETLIFY_AUTH_TOKEN: ${{ secrets.NETLIFY_TOKEN }}
          NETLIFY_SITE_ID: ${{ secrets.NETLIFY_SITE_ID }}
        with:
          args: deploy --dir=src/frontend/dist --prod


Overwriting .github/workflows/netlify.yml


# Backend

In [5]:
%%writefile src/backend/package.json
{
  "name": "nfl-kicker-backend",
  "version": "1.0.0",
  "description": "Express backend for NFL Kicker Assessment",
  "main": "server.js",
  "type": "module",
  "scripts": {
    "start": "node server.js",
    "dev": "nodemon server.js"
  },
  "dependencies": {
    "cors": "^2.8.5",
    "csv-parser": "^3.0.0",
    "express": "^4.18.2",
    "http-proxy-middleware": "^3.0.5",
    "multer": "^1.4.5-lts.1"
  },
  "devDependencies": {
    "nodemon": "^3.0.2"
  },
  "keywords": [
    "nfl",
    "kicker",
    "assessment",
    "api"
  ],
  "author": "",
  "license": "MIT",
  "engines": {
    "node": ">=18.0.0"
  }
}


Overwriting src/backend/package.json


In [6]:
%%writefile src/backend/server.js
import express from 'express';
import cors from 'cors';
import multer from 'multer';
import csv from 'csv-parser';
import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';
import { createProxyMiddleware } from 'http-proxy-middleware';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);

const app = express();

/* ------------------------------------------------------------------ */
/*  Resolve model-API location                                         */
/* ------------------------------------------------------------------ */
const inDocker = fs.existsSync('/.dockerenv');
const inWSL = !!process.env.WSL_DISTRO_NAME;

const MODEL_API_URL =
  process.env.MODEL_API_URL ||
  (inDocker || inWSL ? 'http://host.docker.internal:8000' : 'http://127.0.0.1:8000');

console.log(`🔗  Proxying /ml/*  →  ${MODEL_API_URL}`);

function getAllowedOrigins() {
  const origins = process.env.ALLOWED_ORIGINS;
  return origins ? origins.split(',') : ['http://localhost:5173'];
}

// Middleware
app.use(cors({
  origin: getAllowedOrigins(),
  credentials: true,
}));

app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// Configure multer for file uploads
const storage = multer.diskStorage({
  destination: (req, file, cb) => {
    const uploadDir = 'uploads/';
    // Ensure uploads directory exists (Railway volume mount point)
    if (!fs.existsSync(uploadDir)) {
      fs.mkdirSync(uploadDir, { recursive: true });
    }
    cb(null, uploadDir);
  },
  filename: (req, file, cb) => {
    // Add timestamp to prevent filename conflicts
    const timestamp = Date.now();
    const originalName = file.originalname;
    cb(null, `${timestamp}-${originalName}`);
  }
});

const upload = multer({
  storage,
  limits: {
    fileSize: 10 * 1024 * 1024, // 10MB limit
    files: 1
  },
  fileFilter: (req, file, cb) => {
    // Only allow CSV files
    if (file.mimetype === 'text/csv' || file.originalname.endsWith('.csv')) {
      cb(null, true);
    } else {
      cb(new Error('Only CSV files are allowed'), false);
    }
  }
});

// ─── ML Model API Proxy ─────────────────────────────────────────────────────
// Prefer explicit IPv4 loopback to avoid IPv6 ↔ IPv4 DNS quirks on Windows/WSL

app.use('/ml', createProxyMiddleware({
  target: MODEL_API_URL,
  changeOrigin: true,
  pathRewrite: { '^/ml': '' },
  // Shorter timeouts so UI can fail fast instead of hanging ~2 min default
  timeout: 15_000,      // 15 s socket inactivity timeout (Node.js)
  proxyTimeout: 15_000, // 15 s upstream response timeout
  onProxyReq: (_, req) => console.log(`[ml] → ${req.method} ${req.url}`),
  onError: (err, req, res) => {
    console.error('[ml] proxy error:', err.code || err.message);
    res.status(502).json({ error: 'Model API unreachable', detail: err.code || err.message });
  },
  onProxyRes: (proxyRes, _, res) => {
    const ct = proxyRes.headers['content-type'] || '';
    if (!ct.includes('application/json')) res.status(502); // enforce JSON to caller
  }
}));

// API Routes

// Health check endpoint
app.get('/api/ping', (req, res) => {
  res.json({
    pong: true,
    timestamp: new Date().toISOString(),
    environment: process.env.NODE_ENV || 'development'
  });
});

// CSV upload and processing endpoint
app.post('/api/upload', upload.single('file'), (req, res) => {
  if (!req.file) {
    return res.status(400).json({ error: 'No file uploaded' });
  }

  const results = [];
  const filePath = req.file.path;

  console.log(`Processing uploaded file: ${req.file.originalname}`);

  // Stream parse the CSV file
  fs.createReadStream(filePath)
    .pipe(csv())
    .on('data', (data) => {
      // Clean and validate data
      const cleanData = {};
      Object.keys(data).forEach(key => {
        const cleanKey = key.trim();
        const value = data[key]?.toString().trim();

        // Convert numeric fields
        if (['player_id', 'distance', 'made', 'week', 'year'].includes(cleanKey)) {
          cleanData[cleanKey] = isNaN(value) ? value : Number(value);
        } else {
          cleanData[cleanKey] = value;
        }
      });

      // Basic validation - ensure required fields are present
      if (cleanData.player_id && cleanData.player_name &&
          cleanData.distance !== undefined && cleanData.made !== undefined) {
        results.push(cleanData);
      }
    })
    .on('end', () => {
      // Clean up the uploaded file
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });

      console.log(`Processed ${results.length} records from ${req.file.originalname}`);

      res.json({
        success: true,
        data: results,
        rowCount: results.length,
        filename: req.file.originalname
      });
    })
    .on('error', (error) => {
      console.error('Error processing CSV:', error);

      // Clean up the uploaded file on error
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });

      res.status(500).json({
        error: 'Error processing CSV file',
        details: error.message
      });
    });
});

// Get processed leaderboard data (if we want to store it server-side)
app.get('/api/leaderboard', (req, res) => {
  // This could be enhanced to return cached/processed leaderboard data
  res.json({
    message: 'Leaderboard endpoint - not implemented yet',
    timestamp: new Date().toISOString()
  });
});

// Error handling middleware
app.use((error, req, res, next) => {
  console.error('Server error:', error);

  if (error instanceof multer.MulterError) {
    if (error.code === 'LIMIT_FILE_SIZE') {
      return res.status(400).json({ error: 'File too large (max 10MB)' });
    }
    if (error.code === 'LIMIT_UNEXPECTED_FILE') {
      return res.status(400).json({ error: 'Too many files uploaded' });
    }
  }

  res.status(500).json({
    error: 'Internal server error',
    details: process.env.NODE_ENV === 'development' ? error.message : undefined
  });
});

// 404 handler for API routes
app.use('/api/*', (req, res) => {
  res.status(404).json({ error: 'API endpoint not found' });
});

// In production, serve static files from the public directory
if (process.env.NODE_ENV === 'production') {
  const publicPath = path.join(__dirname, 'public');
  app.use(express.static(publicPath));

  // Serve index.html for any non-API routes (SPA fallback)
  app.get('*', (req, res, next) => {
    if (req.path.startsWith('/api/') || req.path.startsWith('/ml/')) {
      return next();
    }
    res.sendFile(path.join(publicPath, 'index.html'));
  });
}

// Start server
const PORT = process.env.PORT || 5000;
const HOST = process.env.HOST || '0.0.0.0';

app.listen(PORT, HOST, () => {
  console.log(`🚀 NFL Kicker API server running on http://${HOST}:${PORT}`);
  console.log(`📁 Environment: ${process.env.NODE_ENV || 'development'}`);
  console.log(`🔧 CORS enabled for: ${process.env.CORS_ORIGIN || (process.env.NODE_ENV === 'production' ? 'production domains' : 'development (localhost:5173, localhost:3000)')}`);
  console.log(`📊 Health check: http://${HOST}:${PORT}/api/ping`);

  // Railway deployment info
  if (process.env.RAILWAY_ENVIRONMENT) {
    console.log(`🚂 Railway Environment: ${process.env.RAILWAY_ENVIRONMENT}`);
    console.log(`🌍 Railway Service: ${process.env.RAILWAY_SERVICE_NAME || 'Unknown'}`);
  }
});

// Graceful shutdown
process.on('SIGTERM', () => {
  console.log('SIGTERM received, shutting down gracefully');
  process.exit(0);
});

process.on('SIGINT', () => {
  console.log('SIGINT received, shutting down gracefully');
  process.exit(0);
});



Overwriting src/backend/server.js


#  Frontend

In [7]:
%%writefile src/frontend/package.json
{
  "name": "nfl-kicker-frontend",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "start": "serve -s dist -l $PORT",
    "lint": "eslint . --ext js,jsx --report-unused-disable-directives --max-warnings 0",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "recharts": "^2.12.7",
    "lucide-react": "^0.436.0"
  },
  "devDependencies": {
    "@types/react": "^18.2.43",
    "@types/react-dom": "^18.2.17",
    "@vitejs/plugin-react": "^4.2.1",
    "autoprefixer": "^10.4.16",
    "eslint": "^8.55.0",
    "eslint-plugin-react": "^7.33.2",
    "eslint-plugin-react-hooks": "^4.6.0",
    "eslint-plugin-react-refresh": "^0.4.5",
    "postcss": "^8.4.32",
    "tailwindcss": "^3.3.6",
    "vite": "^5.0.8"
  },
  "engines": {
    "node": ">=18.0.0"
  }
}


Overwriting src/frontend/package.json


In [8]:
%%writefile src/frontend/vite.config.js
import { defineConfig, loadEnv } from 'vite'
import react from '@vitejs/plugin-react'

// https://vitejs.dev/config/
export default defineConfig(({ mode }) => {
  const env = loadEnv(mode, process.cwd(), '')
  const API = env.VITE_API_URL || 'http://localhost:5000'   // dev fallback

  return {
    plugins: [react()],
    define: {
      __API_URL__: JSON.stringify(API),                    // usable in code
    },
    server: {
      host: '0.0.0.0',
      port: 5173,
      proxy: {
        '/ml': {
          target: API,           // still hits Express in dev
          changeOrigin: true,
          secure: false,
          xfwd: true,
          onError(err, req) {
            console.error('⚠️ Vite→Express proxy error:', err.code || err.message)
          },
        },
      },
    },
    build: { outDir: 'dist' },
  }
})






Overwriting src/frontend/vite.config.js


#sample front end before we update the nfl frontend in original

In [9]:
%%writefile src/frontend/src/App.jsx
import React, { useState, useEffect } from 'react';
import { LineChart, Line, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ResponsiveContainer, BarChart, Bar, ScatterChart, Scatter, PieChart, Pie, Cell } from 'recharts';
import { Activity, Brain, Database, TrendingUp, AlertCircle, CheckCircle, Play, Upload, Download, Settings, BarChart3, Target } from 'lucide-react';

const MLModelFrontend = () => {
  const [activeTab, setActiveTab] = useState('dashboard');
  const [selectedDataset, setSelectedDataset] = useState('iris');
  const [modelStatus, setModelStatus] = useState('idle');
  const [predictions, setPredictions] = useState([]);
  const [trainingHistory, setTrainingHistory] = useState([]);
  const [apiHealth, setApiHealth] = useState('unknown');
  const [inputData, setInputData] = useState({});
  const [modelMetrics, setModelMetrics] = useState(null);
  const [predictionResults, setPredictionResults] = useState(null);

  // API Base URL - adjust this to match your backend
  const API_BASE = '' // unused – kept for backward compat

  // --- Real API helper  ---------------------------------------------------
  const callApi = async (path, payload = null, opts = {}) => {
    /*
      All frontend→backend traffic is proxied via Express so we only prepend
      the `/ml` prefix.  The Express dev-server then forwards the request to
      FastAPI.  This avoids any CORS issues in local dev & Netlify preview.
    */
    const url = `/ml${path}`

    const cfg = payload !== null
      ? {
          method: 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify(payload),
          ...opts,
        }
      : { method: 'GET', ...opts }

    const res = await fetch(url, cfg)
    if (!res.ok) {
      const txt = await res.text()
      throw new Error(`API ${res.status} — ${txt}`)
    }
    return res.json()
  }

  // Dataset configurations
  const datasets = {
    iris: {
      name: 'Iris Dataset',
      description: 'Classic iris flower classification (Setosa vs Others)',
      features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
      featureLabels: ['Sepal Length (cm)', 'Sepal Width (cm)', 'Petal Length (cm)', 'Petal Width (cm)'],
      targetClasses: ['Setosa', 'Non-Setosa'],
      color: '#8884d8'
    },
    breast_cancer: {
      name: 'Breast Cancer Dataset',
      description: 'Breast cancer diagnosis prediction (Malignant vs Benign)',
      features: ['mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area', 'mean_smoothness'],
      featureLabels: ['Mean Radius', 'Mean Texture', 'Mean Perimeter', 'Mean Area', 'Mean Smoothness'],
      targetClasses: ['Malignant', 'Benign'],
      color: '#82ca9d'
    }
  };

  // Check API health
  useEffect(() => {
    const checkHealth = async () => {
      try {
        const response = await callApi('/health');
        setApiHealth(response.status);
      } catch (error) {
        setApiHealth('error');
      }
    };
    
    checkHealth();
    const interval = setInterval(checkHealth, 30000);
    return () => clearInterval(interval);
  }, []);

  // Initialize input data when dataset changes
  useEffect(() => {
    const initData = {};
    datasets[selectedDataset].features.forEach(feature => {
      initData[feature] = '';
    });
    setInputData(initData);
  }, [selectedDataset]);

  // Train model
  const handleTrainModel = async () => {
    setModelStatus('training');
    try {
      const result = await callApi('/train', {
        method: 'POST',
        body: JSON.stringify({ dataset: selectedDataset })
      });
      
      setModelMetrics(result.metrics);
      setTrainingHistory(prev => [...prev, {
        timestamp: new Date().toISOString(),
        dataset: selectedDataset,
        accuracy: result.metrics.accuracy,
        training_time: result.training_time
      }]);
      
      setModelStatus('trained');
    } catch (error) {
      setModelStatus('error');
    }
  };

  // Make prediction
  const handlePredict = async () => {
    try {
      const features = Object.values(inputData).map(val => parseFloat(val) || 0);
      let result
      if (selectedDataset === 'iris') {
        const payload = {
          rows: [
            {
              sepal_length: parseFloat(inputData.sepal_length) || 0,
              sepal_width: parseFloat(inputData.sepal_width) || 0,
              petal_length: parseFloat(inputData.petal_length) || 0,
              petal_width: parseFloat(inputData.petal_width) || 0,
            },
          ],
        }
        result = await callApi('/predict/iris/rf', payload)
        result.class_name = result.predictions[0] === 0 ? 'Setosa' : 'Non-Setosa'
        result.probability = 1
        result.confidence = 1
      } else {
        const values = features
        const payload = { rows: [{ values }], posterior_samples: 100 }
        result = await callApi('/predict/cancer/bayes', payload)
        result.class_name = result.predictions[0] > 0.5 ? 'Malignant' : 'Benign'
        result.probability = result.predictions[0]
        result.confidence = 1
      }
      
      setPredictionResults(result);
      setPredictions(prev => [...prev, {
        id: Date.now(),
        input: { ...inputData },
        result: result,
        timestamp: new Date().toISOString()
      }]);
    } catch (error) {
      console.error('Prediction error:', error);
    }
  };

  // Generate sample data for visualization
  const generateSampleData = () => {
    return Array.from({ length: 100 }, (_, i) => ({
      x: Math.random() * 10,
      y: Math.random() * 10,
      class: Math.random() > 0.5 ? 'Class A' : 'Class B'
    }));
  };

  const sampleData = generateSampleData();

  // --- Background Bayesian retrain ---------------------------------------
  const handleRetrainBayes = async () => {
    setModelStatus('training')
    try {
      await callApi('/train/cancer/bayes/retrain', { draws: 800, tune: 400 })

      // poll every 10 s until /health indicates the model is loaded
      let loaded = false
      while (!loaded) {
        await new Promise((r) => setTimeout(r, 10_000))
        const h = await callApi('/health')
        loaded = h.models && h.models.cancer_bayes && h.models.cancer_bayes.loaded
      }

      // reload in-memory cache
      await callApi('/models/reload', null, { method: 'POST' })
      setModelStatus('trained')
    } catch (e) {
      console.error(e)
      setModelStatus('error')
    }
  }

  return (
    <div className="min-h-screen bg-gray-50">
      {/* Header */}
      <header className="bg-white shadow-sm border-b">
        <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8">
          <div className="flex justify-between items-center py-4">
            <div className="flex items-center">
              <Brain className="h-8 w-8 text-blue-600 mr-3" />
              <h1 className="text-2xl font-bold text-gray-900">ML Model Dashboard</h1>
            </div>
            <div className="flex items-center space-x-4">
              <div className="flex items-center">
                <div className={`w-3 h-3 rounded-full mr-2 ${
                  apiHealth === 'healthy' ? 'bg-green-500' : 
                  apiHealth === 'error' ? 'bg-red-500' : 'bg-yellow-500'
                }`} />
                <span className="text-sm text-gray-600">API Status: {apiHealth}</span>
              </div>
              <select 
                value={selectedDataset} 
                onChange={(e) => setSelectedDataset(e.target.value)}
                className="px-3 py-2 border border-gray-300 rounded-md focus:outline-none focus:ring-2 focus:ring-blue-500"
              >
                {Object.entries(datasets).map(([key, dataset]) => (
                  <option key={key} value={key}>{dataset.name}</option>
                ))}
              </select>
            </div>
          </div>
        </div>
      </header>

      {/* Navigation */}
      <nav className="bg-white border-b">
        <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8">
          <div className="flex space-x-8">
            {[
              { id: 'dashboard', label: 'Dashboard', icon: Activity },
              { id: 'training', label: 'Training', icon: Brain },
              { id: 'prediction', label: 'Prediction', icon: TrendingUp },
              { id: 'analysis', label: 'Analysis', icon: BarChart3 }
            ].map(({ id, label, icon: Icon }) => (
              <button
                key={id}
                onClick={() => setActiveTab(id)}
                className={`flex items-center px-3 py-4 text-sm font-medium border-b-2 ${
                  activeTab === id
                    ? 'border-blue-500 text-blue-600'
                    : 'border-transparent text-gray-500 hover:text-gray-700'
                }`}
              >
                <Icon className="h-4 w-4 mr-2" />
                {label}
              </button>
            ))}
          </div>
        </div>
      </nav>

      {/* Main Content */}
      <main className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 py-8">
        {/* Dashboard Tab */}
        {activeTab === 'dashboard' && (
          <div className="space-y-6">
            {/* Dataset Info Card */}
            <div className="bg-white rounded-lg shadow p-6">
              <h2 className="text-lg font-semibold mb-4">Current Dataset: {datasets[selectedDataset].name}</h2>
              <p className="text-gray-600 mb-4">{datasets[selectedDataset].description}</p>
              <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
                <div className="flex items-center">
                  <Database className="h-5 w-5 text-blue-500 mr-2" />
                  <span className="text-sm">Features: {datasets[selectedDataset].features.length}</span>
                </div>
                <div className="flex items-center">
                  <Target className="h-5 w-5 text-green-500 mr-2" />
                  <span className="text-sm">Classes: {datasets[selectedDataset].targetClasses.join(', ')}</span>
                </div>
                <div className="flex items-center">
                  <Settings className="h-5 w-5 text-purple-500 mr-2" />
                  <span className="text-sm">Model: Bayesian LogReg</span>
                </div>
              </div>
            </div>

            {/* Quick Stats */}
            <div className="grid grid-cols-1 md:grid-cols-4 gap-6">
              <div className="bg-white rounded-lg shadow p-6">
                <div className="flex items-center">
                  <CheckCircle className="h-8 w-8 text-green-500" />
                  <div className="ml-4">
                    <p className="text-sm text-gray-600">Model Status</p>
                    <p className="text-lg font-semibold capitalize">{modelStatus}</p>
                  </div>
                </div>
              </div>
              <div className="bg-white rounded-lg shadow p-6">
                <div className="flex items-center">
                  <TrendingUp className="h-8 w-8 text-blue-500" />
                  <div className="ml-4">
                    <p className="text-sm text-gray-600">Predictions Made</p>
                    <p className="text-lg font-semibold">{predictions.length}</p>
                  </div>
                </div>
              </div>
              <div className="bg-white rounded-lg shadow p-6">
                <div className="flex items-center">
                  <Activity className="h-8 w-8 text-purple-500" />
                  <div className="ml-4">
                    <p className="text-sm text-gray-600">Training Runs</p>
                    <p className="text-lg font-semibold">{trainingHistory.length}</p>
                  </div>
                </div>
              </div>
              <div className="bg-white rounded-lg shadow p-6">
                <div className="flex items-center">
                  <BarChart3 className="h-8 w-8 text-orange-500" />
                  <div className="ml-4">
                    <p className="text-sm text-gray-600">Accuracy</p>
                    <p className="text-lg font-semibold">
                      {modelMetrics ? `${(modelMetrics.accuracy * 100).toFixed(1)}%` : 'N/A'}
                    </p>
                  </div>
                </div>
              </div>
            </div>

            {/* Recent Activity */}
            <div className="bg-white rounded-lg shadow p-6">
              <h3 className="text-lg font-semibold mb-4">Recent Activity</h3>
              <div className="space-y-3">
                {[...trainingHistory, ...predictions].slice(-5).map((item, index) => (
                  <div key={index} className="flex items-center justify-between py-2 border-b border-gray-100">
                    <div className="flex items-center">
                      {item.dataset ? (
                        <Brain className="h-4 w-4 text-blue-500 mr-2" />
                      ) : (
                        <TrendingUp className="h-4 w-4 text-green-500 mr-2" />
                      )}
                      <span className="text-sm">
                        {item.dataset ? `Trained ${item.dataset} model` : 'Made prediction'}
                      </span>
                    </div>
                    <span className="text-xs text-gray-500">
                      {new Date(item.timestamp).toLocaleTimeString()}
                    </span>
                  </div>
                ))}
              </div>
            </div>
          </div>
        )}

        {/* Training Tab */}
        {activeTab === 'training' && (
          <div className="space-y-6">
            <div className="bg-white rounded-lg shadow p-6">
              <h2 className="text-lg font-semibold mb-4">Model Training</h2>
              <div className="mb-6">
                <button
                  onClick={handleTrainModel}
                  disabled={modelStatus === 'training'}
                  className={`flex items-center px-4 py-2 rounded-md ${
                    modelStatus === 'training'
                      ? 'bg-gray-400 cursor-not-allowed'
                      : 'bg-blue-600 hover:bg-blue-700'
                  } text-white`}
                >
                  <Play className="h-4 w-4 mr-2" />
                  {modelStatus === 'training' ? 'Training...' : 'Train Model'}
                </button>

                {/* NEW – retrain Bayesian button */}
                {selectedDataset === 'breast_cancer' && (
                  <button
                    onClick={handleRetrainBayes}
                    disabled={modelStatus === 'training'}
                    className={`ml-4 flex items-center px-4 py-2 rounded-md ${
                      modelStatus === 'training'
                        ? 'bg-gray-400 cursor-not-allowed'
                        : 'bg-orange-600 hover:bg-orange-700'
                    } text-white`}
                  >
                    <Upload className="h-4 w-4 mr-2" />
                    Retrain Cancer Model
                  </button>
                )}
              </div>

              {modelMetrics && (
                <div className="grid grid-cols-2 md:grid-cols-4 gap-4 mb-6">
                  {Object.entries(modelMetrics).map(([metric, value]) => (
                    <div key={metric} className="text-center p-4 bg-gray-50 rounded-lg">
                      <p className="text-sm text-gray-600 capitalize">{metric.replace('_', ' ')}</p>
                      <p className="text-lg font-semibold">{(value * 100).toFixed(1)}%</p>
                    </div>
                  ))}
                </div>
              )}

              {trainingHistory.length > 0 && (
                <div className="h-64">
                  <h3 className="text-md font-semibold mb-2">Training History</h3>
                  <ResponsiveContainer width="100%" height="100%">
                    <LineChart data={trainingHistory}>
                      <CartesianGrid strokeDasharray="3 3" />
                      <XAxis dataKey="timestamp" tickFormatter={(value) => new Date(value).toLocaleTimeString()} />
                      <YAxis />
                      <Tooltip labelFormatter={(value) => new Date(value).toLocaleString()} />
                      <Legend />
                      <Line type="monotone" dataKey="accuracy" stroke="#8884d8" name="Accuracy" />
                    </LineChart>
                  </ResponsiveContainer>
                </div>
              )}
            </div>
          </div>
        )}

        {/* Prediction Tab */}
        {activeTab === 'prediction' && (
          <div className="space-y-6">
            <div className="bg-white rounded-lg shadow p-6">
              <h2 className="text-lg font-semibold mb-4">Make Prediction</h2>
              <div className="grid grid-cols-1 md:grid-cols-2 gap-4 mb-6">
                {datasets[selectedDataset].features.map((feature, index) => (
                  <div key={feature}>
                    <label className="block text-sm font-medium text-gray-700 mb-1">
                      {datasets[selectedDataset].featureLabels[index]}
                    </label>
                    <input
                      type="number"
                      step="0.1"
                      value={inputData[feature] || ''}
                      onChange={(e) => setInputData(prev => ({ ...prev, [feature]: e.target.value }))}
                      className="w-full px-3 py-2 border border-gray-300 rounded-md focus:outline-none focus:ring-2 focus:ring-blue-500"
                      placeholder={`Enter ${feature}`}
                    />
                  </div>
                ))}
              </div>
              <button
                onClick={handlePredict}
                className="flex items-center px-4 py-2 bg-green-600 hover:bg-green-700 text-white rounded-md"
              >
                <TrendingUp className="h-4 w-4 mr-2" />
                Predict
              </button>

              {predictionResults && (
                <div className="mt-6 p-4 bg-blue-50 rounded-lg">
                  <h3 className="font-semibold mb-2">Prediction Results</h3>
                  <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
                    <div>
                      <p className="text-sm text-gray-600">Predicted Class</p>
                      <p className="text-lg font-semibold">{predictionResults.class_name}</p>
                    </div>
                    <div>
                      <p className="text-sm text-gray-600">Probability</p>
                      <p className="text-lg font-semibold">{(predictionResults.probability * 100).toFixed(1)}%</p>
                    </div>
                    <div>
                      <p className="text-sm text-gray-600">Confidence</p>
                      <p className="text-lg font-semibold">{(predictionResults.confidence * 100).toFixed(1)}%</p>
                    </div>
                  </div>
                </div>
              )}
            </div>

            {predictions.length > 0 && (
              <div className="bg-white rounded-lg shadow p-6">
                <h3 className="text-lg font-semibold mb-4">Prediction History</h3>
                <div className="overflow-x-auto">
                  <table className="min-w-full table-auto">
                    <thead>
                      <tr className="bg-gray-50">
                        <th className="px-4 py-2 text-left text-sm font-medium text-gray-700">Timestamp</th>
                        <th className="px-4 py-2 text-left text-sm font-medium text-gray-700">Prediction</th>
                        <th className="px-4 py-2 text-left text-sm font-medium text-gray-700">Confidence</th>
                      </tr>
                    </thead>
                    <tbody>
                      {predictions.slice(-10).map((pred) => (
                        <tr key={pred.id} className="border-b border-gray-200">
                          <td className="px-4 py-2 text-sm text-gray-600">
                            {new Date(pred.timestamp).toLocaleString()}
                          </td>
                          <td className="px-4 py-2 text-sm font-medium">
                            {pred.result.class_name}
                          </td>
                          <td className="px-4 py-2 text-sm">
                            {(pred.result.confidence * 100).toFixed(1)}%
                          </td>
                        </tr>
                      ))}
                    </tbody>
                  </table>
                </div>
              </div>
            )}
          </div>
        )}

        {/* Analysis Tab */}
        {activeTab === 'analysis' && (
          <div className="space-y-6">
            <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
              <div className="bg-white rounded-lg shadow p-6">
                <h3 className="text-lg font-semibold mb-4">Feature Distribution</h3>
                <div className="h-64">
                  <ResponsiveContainer width="100%" height="100%">
                    <BarChart data={datasets[selectedDataset].features.map((feature, index) => ({
                      name: feature,
                      value: Math.random() * 100,
                      fill: `hsl(${index * 60}, 70%, 50%)`
                    }))}>
                      <CartesianGrid strokeDasharray="3 3" />
                      <XAxis dataKey="name" angle={-45} textAnchor="end" height={100} />
                      <YAxis />
                      <Tooltip />
                      <Bar dataKey="value" />
                    </BarChart>
                  </ResponsiveContainer>
                </div>
              </div>

              <div className="bg-white rounded-lg shadow p-6">
                <h3 className="text-lg font-semibold mb-4">Class Distribution</h3>
                <div className="h-64">
                  <ResponsiveContainer width="100%" height="100%">
                    <PieChart>
                      <Pie
                        data={datasets[selectedDataset].targetClasses.map((className, index) => ({
                          name: className,
                          value: 50 + Math.random() * 50,
                          fill: ['#8884d8', '#82ca9d', '#ffc658', '#ff7300'][index % 4]
                        }))}
                        cx="50%"
                        cy="50%"
                        labelLine={false}
                        label={({ name, percent }) => `${name}: ${(percent * 100).toFixed(0)}%`}
                        outerRadius={80}
                        fill="#8884d8"
                        dataKey="value"
                      >
                        {datasets[selectedDataset].targetClasses.map((entry, index) => (
                          <Cell key={`cell-${index}`} fill={['#8884d8', '#82ca9d', '#ffc658', '#ff7300'][index % 4]} />
                        ))}
                      </Pie>
                      <Tooltip />
                    </PieChart>
                  </ResponsiveContainer>
                </div>
              </div>
            </div>

            <div className="bg-white rounded-lg shadow p-6">
              <h3 className="text-lg font-semibold mb-4">Data Visualization</h3>
              <div className="h-64">
                <ResponsiveContainer width="100%" height="100%">
                  <ScatterChart data={sampleData}>
                    <CartesianGrid strokeDasharray="3 3" />
                    <XAxis type="number" dataKey="x" />
                    <YAxis type="number" dataKey="y" />
                    <Tooltip cursor={{ strokeDasharray: '3 3' }} />
                    <Scatter
                      name="Class A"
                      data={sampleData.filter(d => d.class === 'Class A')}
                      fill="#8884d8"
                    />
                    <Scatter
                      name="Class B"
                      data={sampleData.filter(d => d.class === 'Class B')}
                      fill="#82ca9d"
                    />
                    <Legend />
                  </ScatterChart>
                </ResponsiveContainer>
              </div>
            </div>
          </div>
        )}
      </main>
    </div>
  );
};

export default MLModelFrontend;


Overwriting src/frontend/src/App.jsx


# Validation

In [10]:
%%writefile src/scripts/ci-validate.js
#!/usr/bin/env node

function validateEnvVars() {
  const apiUrl = process.env.VITE_API_URL;
  const allowedOrigins = process.env.ALLOWED_ORIGINS;
  const corsOrigin = allowedOrigins;
  const modelApiUrl = process.env.MODEL_API_URL;

  if (!apiUrl) {
    console.error('❌ VITE_API_URL is not set');
    process.exit(1);
  }

  if (!allowedOrigins) {
    console.error('❌ ALLOWED_ORIGINS is not set');
    process.exit(1);
  }

  if (!process.env.MODEL_API_URL && process.env.NODE_ENV === 'production') {
    console.error('❌ MODEL_API_URL is not set for production build');
    process.exit(1);
  }

  try {
    const apiDomain = new URL(apiUrl).hostname;
    const corsList = allowedOrigins.split(',').map(o => o.trim());

    // Check if the API domain is allowed by CORS_ORIGIN
    const isAllowed = corsList.some(origin => {
      if (origin.startsWith('/') && origin.endsWith('/')) {
        // It's a regex pattern
        const regex = new RegExp(origin.slice(1, -1));
        return regex.test(apiDomain);
      }
      return new URL(origin).hostname === apiDomain;
    });

    if (!isAllowed) {
      console.error(`❌ API domain ${apiDomain} is not allowed by ALLOWED_ORIGINS`);
      console.error(`   ALLOWED_ORIGINS: ${allowedOrigins}`);
      process.exit(1);
    }

    console.log('✅ Environment variables are consistent');
    process.exit(0);
  } catch (error) {
    console.error('❌ Invalid URL format in environment variables');
    console.error(error);
    process.exit(1);
  }
}

validateEnvVars();



Overwriting src/scripts/ci-validate.js
